**Import libraries**

In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from torch.backends import cudnn

from torchvision import transforms

from tqdm import tqdm

In [2]:
import wandb
wandb.init(project="aml-lab03", entity="peiro98", name="aml-lab03")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: peiro98 (use `wandb login --relogin` to force relogin)


**Set Arguments**

In [3]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"  # 'cuda' or 'cpu'

# available classes: "dog", "elephant", "giraffe", "guitar", "horse", "house", "person"
NUM_CLASSES = 7

BATCH_SIZE = 4
LR = 5e-4
MOMENTUM = 0.9
WEIGHT_DECAY = 5e-5

NUM_EPOCHS = 50
STEP_SIZE = 10
GAMMA = 0.1

LOG_FREQUENCY = 10
TRAIN_RATIO = 0.75

**Define Data Preprocessing**

In [4]:
# Define transforms for training phase
train_transform = transforms.Compose(
    [
        # 227x227 -> 224x224
        transforms.CenterCrop(224),
        # convert to tensor
        transforms.ToTensor(),
        # normalizes tensor with mean and standard deviation
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ]
)

# Define transforms for the evaluation phase
eval_transform = train_transform

### Dataset loading

#### Source dataset

In [5]:
DATA_DIR = "PACS/data"
SRC_DATA_DIR = f"{DATA_DIR}/photo/"

from PACS.PACS import PACSDataset

# Prepare Pytorch train/test Datasets
src_dataset = PACSDataset(SRC_DATA_DIR, transform=train_transform)

# take the indicies corresponding to train samples
src_train_indices = np.random.choice(len(src_dataset), size=int(1.0 * len(src_dataset)))
# src_val_indices = np.setdiff1d(np.arange(len(src_dataset)), src_train_indices)

src_train_dataset = Subset(src_dataset, src_train_indices)
# src_validation_dataset = Subset(src_dataset, src_val_indices)

print("[SRC] train size:", len(src_train_dataset))
# print("[SRC] validation size:", len(src_validation_dataset))

[SRC] train size: 1670


#### Target dataset

In [6]:
TARGET_DATA_DIR = f"{DATA_DIR}/art_painting/"

# Prepare Pytorch train/test Datasets
target_dataset = PACSDataset(TARGET_DATA_DIR, transform=train_transform)

# take the indicies corresponding to train samples
target_train_indices = np.random.choice(
    len(target_dataset), size=int(TRAIN_RATIO * len(target_dataset))
)
target_val_indices = np.setdiff1d(np.arange(len(target_dataset)), target_train_indices)

target_train_dataset = Subset(target_dataset, target_train_indices)
target_validation_dataset = Subset(target_dataset, target_val_indices)

print("[TARGET] train size:", len(target_train_dataset))
print("[TARGET] validation size:", len(target_validation_dataset))

[TARGET] train size: 1536
[TARGET] validation size: 972


**Prepare Dataloaders**

In [7]:
# Train dataloaders
src_dataloader = DataLoader(
    src_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True
)
# src_train_dataloader = DataLoader(src_train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
# target_train_dataloader = DataLoader(target_train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)

# Validation dataloaders
target_dataloader = DataLoader(
    target_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, drop_last=True
)
# src_validation_dataloader = DataLoader(src_validation_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, drop_last=True)
# target_validation_dataloader = DataLoader(target_validation_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, drop_last=True)

**Prepare Network**

In [8]:
from DANN import build_model as build_DANN

DANN = build_DANN(7, 2, True)  # n. of classes, n. of domains, pretrained

**Prepare Training**

In [9]:
# classification => cross entropy loss
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(
    DANN.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY
)

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

**Train**

In [10]:
# By default, everything is loaded to cpu
DANN = DANN.to(DEVICE)  # this will bring the network to GPU if DEVICE is cuda
wandb.watch(DANN, log_freq=10)

cudnn.benchmark = True

current_step = 0
# Start iterating over the epochs
for epoch in tqdm(range(10)):
    # print('Starting epoch {}/{}, LR = {}'.format(epoch+1, NUM_EPOCHS, scheduler.get_last_lr()))

    # Iterate over the dataset
    for images, labels in src_dataloader:
        # Bring data over the device of choice
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)

        DANN.train()  # Sets module in training mode

        optimizer.zero_grad()  # Zero-ing the gradients
        outputs = DANN(images)

        # Compute loss based on output and ground truth
        loss = criterion(outputs, labels)

        # Log loss
        # if current_step % LOG_FREQUENCY == 0:
        #  print('Step {}, Loss {}'.format(current_step, loss.item()))

        # Compute gradients for each layer and update weights
        loss.backward()  # backward pass: computes gradients
        optimizer.step()  # update weights based on accumulated gradients

        current_step += 1

    # Step the scheduler
    scheduler.step()

  0%|          | 0/10 [00:02<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 64.00 MiB (GPU 0; 1.96 GiB total capacity; 1.07 GiB already allocated; 44.88 MiB free; 1.16 GiB reserved in total by PyTorch)

**Validation**

In [11]:
DANN.train(False)  # Set Network to evaluation mode

running_corrects = 0
for images, labels in tqdm(target_dataloader):
    images = images.to(DEVICE)
    labels = labels.to(DEVICE)

    # Forward Pass
    outputs = DANN(images)

    # Get predictions
    _, preds = torch.max(outputs.data, 1)

    # Update Corrects
    running_corrects += torch.sum(preds == labels.data).data.item()

# Calculate Accuracy
accuracy = running_corrects / float(len(target_dataset))

print("Validation Accuracy: {}".format(accuracy))

100%|██████████| 89/89 [00:05<00:00, 16.60it/s]

Validation Accuracy: 0.8237347294938918


**Test**

In [12]:
net = net.to(DEVICE)  # this will bring the network to GPU if DEVICE is cuda
net.train(False)  # Set Network to evaluation mode

running_corrects = 0
for images, labels in tqdm(test_dataloader):
    images = images.to(DEVICE)
    labels = labels.to(DEVICE)

    # Forward Pass
    outputs = net(images)

    # Get predictions
    _, preds = torch.max(outputs.data, 1)

    # Update Corrects
    running_corrects += torch.sum(preds == labels.data).data.item()

# Calculate Accuracy
accuracy = running_corrects / float(len(test_dataset))

print("Test Accuracy: {}".format(accuracy))

100%|██████████| 91/91 [00:05<00:00, 17.24it/s]

Test Accuracy: 0.8354649153128241
